# Load full data 

In [2]:
import numpy as np
import pandas as pd
from scipy import special
from copy import copy
import math
import seaborn as sns
import importlib
import matplotlib.pyplot as plt
import warnings
from scipy.stats import norm
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [3]:
df_full = pd.read_csv('/Users/harshparikh/IIC/step1_output/covariates-full.csv',index_col=0)

In [4]:
df_full.head()

,Gender,Age,The start day of first EEG,Total EEG,Date of admission,Date of discharge,marrital status unmarried (if the pt was divorced or is widow is still in this category)=0 married=1,iGCS-Total,iGCS = T? no=0 yes=1,iGCS-E,...,HAI-Cdiff No=0 Yes=1,DVT No=0 Yes=1,"New trach (Date) DC summary, QPID, no need=0, if yes= mention the date","New Peg (Date) DC summary, QPID","Tx hypothermia (induced hypothermia or therapeutic hypothermia), DC SUMMARY, QPID No=0 Yes=1",Final Neuro Dx ischemic stroke =1 hemorrhagic stroke=2 SAH=3 SDH=4 SDH+other TBI (including SAH)=5 Other TBI (including SAH)=6 Seizures/status epilepticus=7 Brain tumor/primary or mets=8 CNS infection/inflammation=9 Hypoxic ischemic encephalopathy/Anoxic brain injury=10 Toxic metabolic encephalopathy=11 Other neurosurgical (non tumor/trauma) (eg EVD/VPS etc)=12 Primary psychiatric disorder=13 Other structural-degenerative diseases=18 Spells=19,"Primary systemic Dx none=0 Respiratory disorders=1 cardiovascular disorders=2 Renal failure=3 Liver disorders=4 Gastrointestinal non-hemorrhagic=5 Genitourinary=6 Systemic hemorrhage=7 Endocrine emergency=8 Non head trauma (including spine)=9 Malignancy (solid tumors/hematologic)=10 Other post operative (eg CT surgery, transplant)=11 Primary hematological disorder=12 Immunological=13 Dermatological/Musculoskeletal=14",Primary systemic dx Sepsis/Shock No=0 Yes=1,External Ventricular Drain (EVD) No=0 Yes=1,BOLT N0=0 Yes=1
Index,,,,,,,,,,,,,,,,,,,,,
sid1,F,19.93,2015-07-15 16:37:00,66.64,2015-07-14 00:00:00,2015-07-18 00:00:00,0.0,11,0,4,...,0,0,NaN,NaN,NaN,19,"2,14",0,0,0
sid2,F,85.01,2015-07-14 13:07:00,40.72,2015-07-03 00:00:00,2015-08-05 00:00:00,0.0,7,0,1,...,0,0,2015-07-07 00:00:00,2015-07-07 00:00:00,NaN,2,"1,2",0,1,0
sid3,M,45.91,2015-07-14 13:05:00,46.66,2015-07-12 00:00:00,2015-09-14 00:00:00,1.0,8,0,3,...,0,0,NaN,NaN,NaN,10,"1,14",0,0,0
sid4,M,53.99,2015-07-13 11:56:00,26.63,2015-07-10 00:00:00,2015-07-24 00:00:00,1.0,4,0,1,...,0,0,NaN,NaN,NaN,4,"1,9",0,1,1
sid5,M,23.08,2015-07-08 03:51:00,32.97,2015-07-07 00:00:00,2015-07-13 00:00:00,0.0,6,1,1,...,0,0,NaN,NaN,NaN,"7,13",0,0,0,0
